In [ ]:
#This take in a a lsit of LSTMs for Time and Date and outputs a forcast of the time it would take and the demand

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
import os
from random import *
import pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

os.chdir('C:\\Users\\Kepa\\Documents\\Github\\Hoffman_Leinwand_COMP755_Project')

ODemand_dataframe = pandas.read_csv('Demand_Data3_Total_CV_2017_with_zeros.csv')
OTime_dataframe = pandas.read_csv('neighbors_2017_combined_train_and_test_with_predictions_fixed_with_zeros.csv')
ODemand_test_datafrmae =  pandas.read_csv('Demand_Data3_Test_Predictions_2017_with_zeros.csv')

print(OTime_dataframe)


Using TensorFlow backend.


         Unnamed: 0  OLS_CV_Prediction  OLS_Prediction  OLS_fitted_value  \
0                 0         406.913122       -1.000000        407.939954   
1                 1         296.322699       -1.000000        296.614283   
2                 2         376.331127       -1.000000        376.436955   
3                 3         387.360999       -1.000000        386.823059   
4                 4         421.656756       -1.000000        423.522370   
5                 5         261.749575       -1.000000        260.728148   
6                 6         363.038591       -1.000000        362.397935   
7                 7         257.651050       -1.000000        257.602779   
8                 8         324.504269       -1.000000        324.761492   
9                 9         295.462329       -1.000000        295.705884   
10               10         246.562657       -1.000000        246.351286   
11               11         320.373957       -1.000000        320.900264   
12          

In [6]:
#Times to estiamte For. For now, this will be randonly generated:
n= 1

#Location The taxi is in
PULocationID  = 4


PU_DO_list = OTime_dataframe['PU_DO']
PU_DO_list = np.unique(PU_DO_list)
PU_DO_list = PU_DO_list.tolist()

PULoc_list = np.unique(ODemand_dataframe['PULocationID'])
PULoc_list = PULoc_list.tolist()

print(PULoc_list)

#Demand Matrix
Demand_Matrix  = np.matrix(np.ones((n,len(PULoc_list))) * 0)
Time_Matrix =  np.matrix(np.ones((len(PULoc_list),len(PULoc_list))) * np.inf)

#Time Matrix
PU_list = [0] * len(PU_DO_list)
DO_list = [0] * len(PU_DO_list)
for i in range(len(PU_DO_list)):
    current_PU_DO = PU_DO_list[i]
    [current_loc,destination]  = current_PU_DO.split('_')
    PU_list[i] = current_loc
    DO_list[i] = destination

#Make a dictionary to map each PU to the index of teh matrix
keys = np.unique(PU_list) 
values = range(len(PU_list))
keys_and_vals = zip(keys,values)
dictionary = {}
for keys, vals in keys_and_vals:
    dictionary[keys] = vals

print(dictionary)    

#Make the Time Matrix




#Chosen Times
chosen_Time_list = [0] * n
for j in range(n):
    chosen_Time_list[j] = randint(0,1461)
    
print(chosen_Time_list)


[4, 12, 13, 24, 43, 45, 48, 50, 68, 79, 87, 88, 90, 100, 107, 113, 114, 125, 137, 140, 141, 142, 143, 144, 148, 151, 158, 161, 162, 163, 164, 170, 186, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 246, 249, 261, 262, 263]
{'100': 0, '107': 1, '113': 2, '114': 3, '12': 4, '125': 5, '13': 6, '137': 7, '140': 8, '141': 9, '142': 10, '143': 11, '144': 12, '148': 13, '151': 14, '158': 15, '161': 16, '162': 17, '163': 18, '164': 19, '170': 20, '186': 21, '209': 22, '211': 23, '224': 24, '229': 25, '230': 26, '231': 27, '232': 28, '233': 29, '234': 30, '236': 31, '237': 32, '238': 33, '239': 34, '24': 35, '246': 36, '249': 37, '261': 38, '262': 39, '263': 40, '4': 41, '43': 42, '45': 43, '48': 44, '50': 45, '68': 46, '79': 47, '87': 48, '88': 49, '90': 50}
[227]


In [3]:
#Demand Matrx
os.chdir('C:\\Users\\Kepa\\Documents\\Github\\Hoffman_Leinwand_COMP755_Project\\LSTM_Demand')
#For each Time
for i in range(n):
    current_time = chosen_Time_list[i]
    
    #For each location
    for j in range(len(PULoc_list)):
        print(j / len(PULoc_list))
        string = 'Demand_LSTM' + str(j) +'.sav'
        model = pickle.load(open(string, 'rb'))
        
        
        dataframe = ODemand_dataframe
        dataframe = dataframe[dataframe['PULocationID'] == PULoc_list[j]]
        dataframe = dataframe['passenger_count']
        dataset = dataframe.values
        dataset = dataset.astype('float32')
        dataset = np.reshape(dataset, (len(dataset),1))
        print(dataset)


        #Scalar transform
        scaler = MinMaxScaler(feature_range=(0, 1))
        dataset = scaler.fit_transform(dataset)
        
        dataframe = ODemand_dataframe
        dataframe = dataframe[dataframe['PULocationID'] ==  PULoc_list[j]]
        dataframe['passenger_count'] = dataset
        dataframe2 = dataframe[dataframe['Month'] > 10]
        dataframe2 = dataframe2['passenger_count']
        dataset2 = dataframe2.values
        dataset2 = dataset2.astype('float32')
        dataset2 = np.reshape(dataset2, (len(dataset2),1))
        test2_size = len(dataset2)
        test2 = dataset2
        

        
        
        def create_dataset(dataset, look_back=1):
            dataX, dataY = [], []
            for i in range(len(dataset)-look_back-1):
                a = dataset[i:(i+look_back), 0]
                dataX.append(a)
                dataY.append(dataset[i + look_back, 0])
            return np.array(dataX), np.array(dataY)

        # reshape into X=t and Y=t+1
        look_back = 1
        test2X, test2Y = create_dataset(test2, look_back)
        #
  
        
        #Reshape it cuz python
        test2X = np.reshape(test2X, (test2X.shape[0], 1, test2X.shape[1]))
   
        #print(test2X[current_time])
        #test2X = test2X[current_time]
        
        
        test2Predict = model.predict(test2X)
        test2Predict = scaler.inverse_transform(test2Predict)

        test2Actual = scaler.inverse_transform([test2Y])
        test2Actual = test2Actual[0]
        
        print('prediction:', test2Predict[current_time])
        print('actual:', test2Actual[current_time])
        Demand_Matrix[i,j] = test2Predict[current_time]
        #print(Demand_Matrix)
        
        if i == 10:
            break
        
        
        
        
        
        

            

0.0
[[135.]
 [144.]
 [190.]
 ...
 [ 69.]
 [ 65.]
 [ 58.]]
prediction: [19.741953]
actual: 11.999999725259837
0.0196078431372549


C:\Users\Kepa\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[[3.]
 [3.]
 [4.]
 ...
 [6.]
 [1.]
 [1.]]
prediction: [13.549451]
actual: 8.0
0.0392156862745098


C:\Users\Kepa\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[[104.]
 [113.]
 [ 79.]
 ...
 [ 83.]
 [ 78.]
 [ 42.]]


...

prediction: [91.63982]
actual: 88.00000378489496
0.9607843137254902
[[202.]
 [215.]
 [139.]
 ...
 [219.]
 [188.]
 [103.]]


C:\Users\Kepa\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


prediction: [267.6649]
actual: 174.0000064661726
0.9803921568627451
[[550.]
 [758.]
 [616.]
 ...
 [481.]
 [352.]
 [256.]]


C:\Users\Kepa\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


prediction: [326.63794]
actual: 280.00001129507973


In [12]:
print(Demand_Matrix)
#np.savetxt("Demand_Matrix.csv", Demand_Matrix, delimiter=",")

[[ 26.29463959  15.11239338 169.59603882  70.02278137 391.77264404
   36.24098206 433.94598389 110.18024445 388.30203247 403.46737671
  117.07459259  53.4262352  300.86016846 303.16931152 368.70874023
  245.78756714 210.15327454  65.85751343 178.15313721 219.1619873
  445.85632324 755.83416748 233.8918457  204.71954346 190.70774841
  240.95036316 176.82429504 576.69512939 518.50622559 466.82992554
  358.80096436 370.52603149 511.87124634  19.16370773 209.74319458
   40.69172668 281.11181641 486.07925415 303.32141113  26.89764023
  174.37243652 591.50018311 770.9050293  880.12011719 348.7253418
  642.72021484 236.21014404 325.78552246 167.02337646 216.71543884
  340.3394165 ]]


In [15]:
print(Time_Matrix)

[[inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 ...
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]
 [inf inf inf ... inf inf inf]]


In [16]:
#Time Matirx
os.chdir('C:\\Users\\Kepa\\Documents\\Github\\Hoffman_Leinwand_COMP755_Project\\LSTM_Time')
#For each Time
for i in range(n):
    current_time = chosen_Time_list[i]
    
    #For each location
    for j in range(len(PU_DO_list)):
        print(j / len(PU_DO_list))
        string = 'Time_LSTM' + str(j) +'.sav'
        model = pickle.load(open(string, 'rb'))
        
        
        dataframe = OTime_dataframe
        dataframe = dataframe[dataframe['PU_DO'] == PU_DO_list[j]]
        dataframe = dataframe['time_delta']
        dataset = dataframe.values
        dataset = dataset.astype('float32')
        dataset = np.reshape(dataset, (len(dataset),1))


        #Scalar transform
        scaler = MinMaxScaler(feature_range=(0, 1))
        dataset = scaler.fit_transform(dataset)
        
        dataframe = OTime_dataframe
        dataframe = dataframe[dataframe['PU_DO'] ==  PU_DO_list[j]]
        dataframe['time_delta'] = dataset
        dataframe2 = dataframe[dataframe['True_Month'] > 10]
        
        current_locs = dataframe2.iloc[[current_time]]
        current_locs = current_locs['PU_DO'].values
        current_locs = current_locs.tolist()
        current_locs = str(current_locs)
        
        [current_loc,current_destination]  = current_locs.split('_')
        current_loc = current_loc[2:]
        current_destination = current_destination[:-2]
        dataframe2 = dataframe2['time_delta']
        dataset2 = dataframe2.values
        dataset2 = dataset2.astype('float32')
        dataset2 = np.reshape(dataset2, (len(dataset2),1))
        test2_size = len(dataset2)
        test2 = dataset2
        
        

        
        
        def create_dataset(dataset, look_back=1):
            dataX, dataY = [], []
            for i in range(len(dataset)-look_back-1):
                a = dataset[i:(i+look_back), 0]
                dataX.append(a)
                dataY.append(dataset[i + look_back, 0])
            return np.array(dataX), np.array(dataY)

        # reshape into X=t and Y=t+1
        look_back = 1
        test2X, test2Y = create_dataset(test2, look_back)
        
        #Reshape it cuz python
        test2X = np.reshape(test2X, (test2X.shape[0], 1, test2X.shape[1]))
        
        
        test2Predict = model.predict(test2X)
        test2Predict = scaler.inverse_transform(test2Predict)
        
        print('current_loc', current_loc)
        print('current_loc', type(current_loc))
        print('x:',dictionary[current_loc])
        print('y:', dictionary[current_destination])
        Time_Matrix[dictionary[current_loc],dictionary[current_destination]] = test2Predict[current_time]
        print(Time_Matrix[dictionary[current_loc],dictionary[current_destination]])
        
        
        
        

            
        
        

0.0


C:\Users\Kepa\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


current_loc 100
current_loc <class 'str'>
x: 0
y: 19
494.6602783203125
0.0043859649122807015


C:\Users\Kepa\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


current_loc 100
current_loc <class 'str'>
x: 0
y: 21
357.5682678222656
0.008771929824561403


...

current_loc 90
current_loc <class 'str'>
x: 50
y: 30
305.00836181640625
0.9912280701754386


C:\Users\Kepa\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


current_loc 90
current_loc <class 'str'>
x: 50
y: 37
259.203857421875
0.9956140350877193


C:\Users\Kepa\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


current_loc 90
current_loc <class 'str'>
x: 50
y: 46
337.237548828125


In [17]:
#np.savetxt("Time_Matrix.csv", Time_Matrix, delimiter=",")
#np.savetxt("Demand_Matrix.csv", Demand_Matrix, delimiter=",")